# Вебинар 3. Коллаборативная фильтрация

# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере iFood. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**

1 - Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- повысить выручку на минимум на 6% за 4 месяца.

2 - По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
    

3 - А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- нам надо всего 5% актуальных e-mail. Обычно это не проблема

4 - Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- e-mail для 5% клиентов, push для для 20%, офлайн клиенты - печатать на чеке

5 - Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- топ-5 товаров

6 - Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- 3 товара по акции на дату отправки письма, из них желательно релевантный товар для конкретного покупателя (если на такой товар есть акция, если нет - просто самые популярные акционные товары); 1 товар - сопутствующий недвавней покупке для стимулирования совершить новую покупку; 1 - аналог недавно просмотренного, но не купленного, при этом по цене чуть дороже просмотренного (повышение среднего чека)

7 - Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- это зависит от бизнеса. При монополии вообще не готовы тратить сверх расходов на коммуникацию, потому что сами придут. При наличии конкуренции - желательно до достижения максимальной эффективности. Например, при тратах на привлечение 10  получаем чистый доход 5 (50% отдача от вложений), при тратах 15 получаем чистый доход 3(20%), а при тратах 20 получаем 1 (5%), то остановится на тратах в 10 будет разумным вариантом.

8 - Cколько мы хотим зарабатывать с одного привлеченного юзера?
- примерно так: потратили на привлечение n юзеров x денег, при этом совершили покупки m юзеров на сумму y. Нам надо (x*n < m*y) на интересную для нас сумму, а это зависит от продаваемого товара. Обычно эту сумму определяет или собственник бизнеса, или кто-то из руководителей.

---

9 - А точно нужно сортировать по вероятности?
- не обязательно. Можно по релевантности, по полуряности. по наличию скидок, по новизне и т.д.

10 - Какую метрику использовать?

11 - Сколько раз в неделю отправляем рассылку?
- зависит от активности пользователя, но я не стал бы заваливать почту. Думаю, раз в 2-4 недели.

12 - В какое время отправляем рассылку?
- в среду в первой половине дня

13 - Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- чередовать содержание рассылки. Сегодня - упор на акции, к праздникам - упор на товары, относящиеся к теме праздника, в другой раз - новинки и т.д.

14 - Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- не обязательно. Разные товары - которые относятся к принципиально разным категориям (детские товары и товары для рыбалки, например)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влязает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

Все эти функции отправим затем в *src.utils*

----